In [2]:
import sys
import os
import pandas as pd
import numpy as np

import gensim
from gensim.models import Word2Vec, KeyedVectors

C:\Users\Dat.Nguyen\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [38]:
from keras.preprocessing import text, sequence
from keras.layers import (Input, Embedding, Bidirectional, LSTM, Dense,
                          Dropout, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D,
                          concatenate)
from keras.models import Model


In [6]:
data_dir = 'data/external/'
train_df = pd.read_csv(data_dir + 'train.csv')
test_df = pd.read_csv(data_dir + 'test.csv')

vocab_size = 100000
max_len = 200
emb_size = 300

In [7]:
vectors = KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
tokenizer = text.Tokenizer(num_words=vocab_size, lower=True)
tokenizer.fit_on_texts(train_df.comment_text.tolist())

In [21]:
X_train = tokenizer.texts_to_sequences(train_df.comment_text)
X_test = tokenizer.texts_to_sequences(test_df.comment_text)

In [21]:
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [25]:
word_index = tokenizer.word_index
embeddings = np.zeros((vocab_size, emb_size))
for word, idx in word_index.items():
    try:
        embeddings[idx] = vectors[word]
    except (KeyError, IndexError):
        continue

In [40]:
input_seq = Input(shape=(max_len,), dtype='int32')
x = Embedding(vocab_size,
            emb_size,
            weights=[embeddings],
            input_length=max_len,
            trainable=False)(input_seq)

x = SpatialDropout1D(0.3)(x)
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(x)

max_pool = GlobalMaxPooling1D()(x)
avg_pool = GlobalAveragePooling1D()(x)
x = concatenate([max_pool, avg_pool])

fc = Dense(6, activation='sigmoid')(x)

In [41]:
nnet = Model(inputs=input_seq, outputs=fc)
nnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
nnet.fit(X_train, train_df.iloc[:, 2:].values, batch_size=32, epochs=1, verbose=1)

Epoch 1/1


    32/159571 [..............................] - ETA: 9:21:21 - loss: 0.6664 - acc: 0.6771

    64/159571 [..............................] - ETA: 5:37:21 - loss: 0.6488 - acc: 0.7734

    96/159571 [..............................] - ETA: 4:08:54 - loss: 0.6330 - acc: 0.8420

   128/159571 [..............................] - ETA: 3:23:18 - loss: 0.6187 - acc: 0.8737

...

159456/159571 [============================>.] - ETA: 3s - loss: 0.0590 - acc: 0.9796

159488/159571 [============================>.] - ETA: 2s - loss: 0.0590 - acc: 0.9795

159520/159571 [============================>.] - ETA: 1s - loss: 0.0590 - acc: 0.9795

159552/159571 [============================>.] - ETA: 0s - loss: 0.0590 - acc: 0.9795

159571/159571 [==============================] - 4378s 27ms/step - loss: 0.0590 - acc: 0.9795


In [50]:
predictions = nnet.predict(X_test, batch_size=32, verbose=1)

    32/153164 [..............................] - ETA: 20:35

    64/153164 [..............................] - ETA: 20:01

    96/153164 [..............................] - ETA: 20:20

   128/153164 [..............................] - ETA: 20:29

   160/153164 [..............................] - ETA: 20:42

...

153056/153164 [============================>.] - ETA: 0s

153088/153164 [============================>.] - ETA: 0s

153120/153164 [============================>.] - ETA: 0s

153152/153164 [============================>.] - ETA: 0s

153164/153164 [==============================] - 1224s 8ms/step


In [50]:
submission = pd.read_csv('data/external/sample_submission.csv')
submission[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] = predictions
submission.to_csv('submission.csv', index=False)